In [2]:
from pathlib import Path
from pathlib import PurePosixPath
from pathlib import PureWindowsPath
from pathlib import PurePath
from pathlib import PosixPath
from pathlib import WindowsPath
import os
import configparser
import cv2
import click

Path is the basic operator for portability.

In [3]:
p = Path('.')
[x for x in p.iterdir() if x.is_dir()]

[PosixPath('.ipynb_checkpoints'), PosixPath('__pycache__')]

In [4]:
list(p.glob('**/*.py'))

[PosixPath('__init__.py'), PosixPath('cfg.py')]

In [5]:
p = Path('/etc')
p

PosixPath('/etc')

In [6]:
q = p / 'init.d' / 'reboot'
q

PosixPath('/etc/init.d/reboot')

In [7]:
q.resolve()

PosixPath('/etc/init.d/reboot')

In [8]:
q.exists()

True

In [9]:
q.is_dir()

False

In [10]:
with q.open() as f: 
    print(f.readline())

#! /bin/sh



Paths are immutable and hashable. Paths of a same flavour are comparable and orderable. These properties respect the flavour’s case-folding semantics:

In [11]:
PurePosixPath('foo') == PurePosixPath('FOO')

False

In [12]:
PureWindowsPath('foo') == PureWindowsPath('FOO')

True

In [13]:
PureWindowsPath('FOO') in { PureWindowsPath('foo') }

True

In [14]:
PureWindowsPath('C:') < PureWindowsPath('d:')

True

Paths of a different flavour compare unequal and cannot be ordered:

In [15]:
PureWindowsPath('foo') == PurePosixPath('foo')

False

In [16]:
try:
    PureWindowsPath('foo') < PurePosixPath('foo')
except TypeError as e:
    if "unorderable types" in str(e):
        print(True)

True


The slash operator helps create child paths, similarly to os.path.join():

In [17]:
p = PurePath('/etc')
p

PurePosixPath('/etc')

In [18]:
p / 'init.d' / 'apache2'

PurePosixPath('/etc/init.d/apache2')

In [19]:
q = PurePath('bin')
'/usr' / q

PurePosixPath('/usr/bin')

The string representation of a path is the raw filesystem path itself (in native form, e.g. with backslashes under Windows), which you can pass to any function taking a file path as a string:

In [20]:
p = PurePath('/etc')
str(p)

'/etc'

In [21]:
bytes(p)

b'/etc'

In [22]:
p = PureWindowsPath('c:/Program Files')
str(p)

'c:\\Program Files'

In [23]:
p = PurePath('/usr/bin/python3')
p

PurePosixPath('/usr/bin/python3')

In [24]:
p.parts

('/', 'usr', 'bin', 'python3')

In [25]:
p = PureWindowsPath('c:/Program Files/PSF')
p.parts

('c:\\', 'Program Files', 'PSF')

In [26]:
PureWindowsPath('c:/Program Files/').drive

'c:'

In [27]:
PureWindowsPath('/Program Files/').drive

''

In [28]:
PurePosixPath('/etc').drive

''

In [29]:
PureWindowsPath('//host/share/foo.txt').drive

'\\\\host\\share'

In [30]:
PureWindowsPath('c:/Program Files/').root

'\\'

In [31]:
PureWindowsPath('c:Program Files/').root

''

In [32]:
PurePosixPath('/etc').root

'/'

In [33]:
PureWindowsPath('//host/share').root

'\\'

In [34]:
PureWindowsPath('c:/Program Files/').anchor

'c:\\'

In [35]:
PureWindowsPath('c:Program Files/').anchor

'c:'

In [36]:
PurePosixPath('/etc').anchor

'/'

In [37]:
PureWindowsPath('//host/share').anchor

'\\\\host\\share\\'

In [38]:
p = PureWindowsPath('c:/foo/bar/setup.py')

In [39]:
p.parents[0]

PureWindowsPath('c:/foo/bar')

In [40]:
p.parents[1]

PureWindowsPath('c:/foo')

In [41]:
p.parents[2]

PureWindowsPath('c:/')

In [42]:
p = PurePosixPath('/a/b/c/d')
p.parent

PurePosixPath('/a/b/c')

In [43]:
p = PurePosixPath('/')

You can only instantiate the class flavour that corresponds to your system (allowing system calls on non-compatible path flavours could lead to bugs or failures in your application):

In [44]:
os.name

'posix'

In [45]:
Path('setup.py')

PosixPath('setup.py')

In [46]:
PosixPath('setup.py')

PosixPath('setup.py')

In [47]:
try:
    WindowsPath('setup.py')
except NotImplementedError as e:
    if "cannot instantiate 'WindowsPath' on your system" in str(e):
        print(True)

True


In [49]:
Path.cwd()

PosixPath('/home/jsk/GitHub/cv2stuff/cv2stuff/notebooks')

In [85]:
p = Path('../../setup.py')
p.exists()

True

In [51]:
p.stat().st_mtime

1456099968.5465758

In [52]:
p.stat().st_mode

33261

In [86]:
p.stat().st_size

1515

In [54]:
p.group()

'jsk'

In [55]:
p.is_dir()

False

In [56]:
p.is_file()

True

In [57]:
p.owner()

'jsk'

In [58]:
setup_text = p.read_text()
'Python' in setup_text

True

In [89]:
PureWindowsPath(home) # I thought this would translate the seperators.

PureWindowsPath('/home/jsk')

In [61]:
ini_file = Path.home() / '.cv2stuff.ini'
ini_file

PosixPath('/home/jsk/.cv2stuff.ini')

In [62]:
ini_file.exists()

False

In [63]:
config = configparser.ConfigParser()

In [64]:
config['DEFAULT'] = {'MAXIMUM_ITERATIONS':'30', 'PIXEL_RESOLUTION':'0.001'}

In [65]:
max = config['DEFAULT'].getint('MAXIMUM_ITERATIONS')
type(max)

int

In [66]:
max

30

In [67]:
pix = max = config['DEFAULT'].getfloat('PIXEL_RESOLUTION')
type(pix)

float

In [68]:
pix

0.001

In [69]:
print(cv2.TERM_CRITERIA_MAX_ITER, "and", cv2.TERM_CRITERIA_EPS)

1 and 2


In [70]:
type(cv2.TERM_CRITERIA_EPS)

int

If the config does not exist then a one time initialization of values must be done.  After that the file can just be read in on startup.

Should default camera calibration values be included for example purposes?

criteria – Termination criteria for the iterative optimization algorithm.

In [71]:
config['criteria'] = {}
criteria = config['criteria']

In [72]:
cv2config_path = Path.home() / '.cv2stuffconf'
cv2config_file = 'default.ini'

In [73]:
config['inner_points'] = {'columns':'9', 'rows':'6'}
config['search_window'] = {'x':'11', 'y':'11'}


In [74]:
if not Path.exists(cv2config_path):
        Path.mkdir(cv2config_path)

In [75]:
cv2config = cv2config_path / cv2config_file
if not Path.exists(cv2config):
    # Create default ini configuration file.
    with open(str(cv2config), 'wt') as configfile:
        config.write(configfile)
    print('Does not exist', cv2config)
else:
    config.read(str(cv2config))

Does not exist /home/jsk/.cv2stuffconf/default.ini


In [76]:
config.sections()

['criteria', 'inner_points', 'search_window']

In [77]:
config_path = click.get_app_dir('cv2stuff')
config_path

'/home/jsk/.config/cv2stuff'

In [78]:
config_name = 'cv2stuff.ini'

In [79]:
Path.exists(Path(config_path))

False

In [80]:
Path.exists(Path(config_path) / config_name)

False

In [81]:
image_path = set(sorted(Path('/home/jsk/GitHub/cv2stuff/data/images').glob('left*[0-9].jpg')))
image_path

{PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left01.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left02.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left03.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left04.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left05.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left06.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left07.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left08.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left09.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left11.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left12.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left13.jpg'),
 PosixPath('/home/jsk/GitHub/cv2stuff/data/images/left14.jpg')}

In [82]:
for p in image_path:
    print(p.parts)

('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left02.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left14.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left06.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left09.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left01.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left12.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left07.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left11.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left13.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left05.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left04.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left08.jpg')
('/', 'home', 'jsk', 'GitHub', 'cv2stuff', 'data', 'images', 'left03.jpg')


In [83]:
parts = ()

In [84]:
if len(parts) == (1 if (False or False) else 0):
    print(True)

True
